In [18]:
import numpy as np
import pandas as pd
from qiskit.quantum_info import SparsePauliOp
from susy_qm import calculate_wz_hamiltonian

In [25]:
# Parameters
N = 2
a = 1.0
c = -0.8
#potential = 'quadratic'
potential = "linear"
boundary_condition = 'dirichlet'
#boundary_condition = 'periodic'

In [26]:
cutoffs = [2,4,8,16]
for cutoff in cutoffs:

    print(cutoff, N, a, potential, boundary_condition, c)
    H = calculate_wz_hamiltonian(cutoff, N, a, potential, boundary_condition, c)
    hamiltonian = SparsePauliOp.from_operator(H)
    num_qubits = hamiltonian.num_qubits
        
    eigenvalues, eigenvectors = np.linalg.eig(H)
    min_eigenvalue = min(eigenvalues.real)
    min_index = np.argmin(eigenvalues)
    min_eigenvalue = eigenvalues[min_index]
    min_eigenvector = np.asarray(eigenvectors[:, min_index])

    active_indices = [i for i, val in enumerate(min_eigenvector) if abs(val) > 1e-8]
    states = [format(i, f'0{num_qubits}b') for i in active_indices]
    print("Active basis states:")
    for s in states:
        print(s)

    print('##########################################')


2 2 1.0 linear dirichlet -0.8
Active basis states:
0100
1000
##########################################
4 2 1.0 linear dirichlet -0.8
Active basis states:
010000
010010
011000
011010
100000
100010
101000
101010
##########################################
8 2 1.0 linear dirichlet -0.8
Active basis states:
01000000
01000010
01000100
01000110
01010000
01010010
01010100
01010110
01100000
01100010
01100100
01100110
01110000
01110010
01110100
10000000
10000010
10000100
10000110
10010000
10010010
10010100
10010110
10100000
10100010
10100100
10100110
10110000
10110010
10110100
##########################################
16 2 1.0 linear dirichlet -0.8
Active basis states:
0100000000
0100000010
0100000100
0100000110
0100001000
0100001010
0100001100
0100100000
0100100010
0100100100
0100100110
0100101000
0100101010
0101000000
0101000010
0101000100
0101000110
0101100000
0101100010
0101100100
0110000000
0110000010
0110100000
0110100010
0111000000
1000000000
1000000010
1000000100
1000000110
1000001000


In [5]:
min_eigenvalue

np.complex128(0.006966011250105146+0j)

In [ ]:
def binary_state(index, num_qubits=num_qubits):
    return format(index, f'0{num_qubits}b')

states = [binary_state(i) for i in active_indices]
print("Active basis states:")
for s in states:
    print(s)

# Transpose to get per-qubit participation
bit_matrix = np.array([[int(bit) for bit in state] for state in states])
qubit_activity = bit_matrix.sum(axis=0)

print("\nQubit activity (number of '1's per qubit):")
for i, count in enumerate(qubit_activity[::-1]):  # reverse to match q2, q1, q0
    print(f"Qubit {i}: {'active' if count > 0 else 'inactive'} ({count} times)")


In [ ]:
def highlight_nonzero(val):
    return 'background-color: green' if val != 0 else ''

df = pd.DataFrame(H.real)
df = df.style.map(highlight_nonzero).format(precision=1)
df


In [ ]:
active_indices = [i for i, val in enumerate(min_eigenvector) if abs(val) > 1e-8]
df = pd.DataFrame(H.real)

def highlight_active_contribution(val, row_idx, col_idx):
    if row_idx in active_indices and col_idx in active_indices:
        return 'background-color: green'
    return ''

styled_df = df.style.apply(
    lambda data: np.vectorize(highlight_active_contribution)(data, *np.indices(data.shape)),
    axis=None
).format("{:.1f}")

#styled_df


In [27]:
states = [format(i, f'0{num_qubits}b') for i in active_indices]
print("Active basis states:")
for s in states:
    print(s)


Active basis states:
0100000000
0100000010
0100000100
0100000110
0100001000
0100001010
0100001100
0100100000
0100100010
0100100100
0100100110
0100101000
0100101010
0101000000
0101000010
0101000100
0101000110
0101100000
0101100010
0101100100
0110000000
0110000010
0110100000
0110100010
0111000000
1000000000
1000000010
1000000100
1000000110
1000001000
1000001010
1000100000
1000100010
1000100100
1000100110
1000101000
1001000000
1001000010
1001000100
1001000110
1001100000
1001100010
1001100100
1010000000
1010000010
1010100000
